# Setup

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import pytest as pt

from datetime import datetime

from car_cost_calculator.running_costs import RunningCosts
from car_cost_calculator.depreciation import TwoStageRate

In [2]:
np.set_printoptions(suppress=True)

In [3]:
# matplotlib configuration
%matplotlib inline
matplotlib.style.use('bmh')

# Common Plotly configuration
py.init_notebook_mode(connected=True)
config={
    'showLink': False,
    'modeBarButtonsToRemove': ['sendDataToCloud', 'hoverClosestCartesian', 'hoverCompareCartesian'],
}

In [4]:
def line_chart(y, x=None):
    py.iplot([{'y':y, 'x':x}], config=config)

In [30]:
actual = RunningCosts(
        initial_vehicle_value=43000,
        initial_vehicle_age=0,
        depreciation_rate=TwoStageRate(
            stage_1_rate=0.23, stage_2_rate=0.1, breakpoint=3),
        years=10,
        km_per_year=15000,
        litres_per_100km=16.0,
        inflation=0.02,
        initial_fuel_price=1.50,
        initial_service_cost=350,
        service_interval_km=10000,
        service_interval_years=1,
        tyre_replacement_interval=40000,
        initial_cost_per_tyre=300)

In [31]:
line_chart(actual.depreciated_value)